In [1]:
!apt install unzip
!mkdir resized
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/resized.zip -d resized/

Streaming output truncated to the last 5000 lines.
  inflating: resized/resized/SPOONBILL/006.jpg  
  inflating: resized/resized/SPOONBILL/007.jpg  
  inflating: resized/resized/SPOONBILL/008.jpg  
  inflating: resized/resized/SPOONBILL/009.jpg  
  inflating: resized/resized/SPOONBILL/010.jpg  
  inflating: resized/resized/SPOONBILL/011.jpg  
  inflating: resized/resized/SPOONBILL/012.jpg  
  inflating: resized/resized/SPOONBILL/013.jpg  
  inflating: resized/resized/SPOONBILL/014.jpg  
  inflating: resized/resized/SPOONBILL/015.jpg  
  inflating: resized/resized/SPOONBILL/016.jpg  
  inflating: resized/resized/SPOONBILL/017.jpg  
  inflating: resized/resized/SPOONBILL/018.jpg  
  inflating: resized/resized/SPOONBILL/019.jpg  
  inflating: resized/resized/SPOONBILL/020.jpg  
  inflating: resized/resized/SPOONBILL/021.jpg  
  inflating: resized/resized/SPOONBILL/022.jpg  
  inflating: resized/resized/SPOONBILL/023.jpg  
  inflating: resized/resized/SPOONBILL/024.jpg  
  inflating: resiz

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize

In [3]:
def load_image_files(container_path, dimension=(64, 64)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = img
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [4]:
image_dataset = load_image_files("resized/resized/")

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

In [6]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23496, 12288) (10070, 12288) (23496,) (10070,)


In [7]:
X_train = X_train/255
X_test = X_test/255

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(verbose = 10)
clf.fit(X_train , y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 61.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 61.3min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=10,
                   warm_start=False)

In [9]:
from sklearn.metrics import accuracy_score

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
print(accuracy_score(y_train , y_pred_train))
accuracy_score(y_test , y_pred_test)

0.6572608103506979


0.1906653426017875